In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from Environment import Environment
from Agent import Agent
import numpy as np
from plot import plot
from Display import disp
from tqdm import tqdm

2023-05-31 03:29:44.985945: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-31 03:29:46.128218: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/danesh/anaconda3/envs/IRS/lib/
2023-05-31 03:29:46.134188: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/danesh/anaconda3/envs/IRS/lib/
2023-05-31 03:29:46.134213: W tensorflow/compiler/tf2t

In [3]:
env = Environment(num_of_antennas=5, num_of_irs1=5, num_of_irs2=5,
                      path_loss_exponent=2, irs1_to_antenna=20,
                      irs2_to_antenna=20, irs1_to_irs2=10, transmitted_power=1)

U1 = env.CreateUser(distance_to_antenna=40, distance_to_irs1=10, distance_to_irs2=20,
                    noise_var=1e-4, los_to_antenna=True, los_to_irs1=True,
                    los_to_irs2=True, sinr_threshold=3, penalty=10, allocated_power=1, weight=1)

U2 = env.CreateUser(distance_to_antenna=40, distance_to_irs1=20, distance_to_irs2=10,
                    noise_var=1e-4, los_to_antenna=True, los_to_irs1=True,
                    los_to_irs2=True, sinr_threshold=3, penalty=10, allocated_power=1, weight=1)

In [7]:
num_of_actions = env.M1 + env.M2 + len(env.Users) * env.N + env.num_of_users - 1

agent = Agent(num_states=env.num_of_users, bound=2, batch_size=128, max_size=100000,
                env=env, n_actions=num_of_actions,
                noise=0.02, alpha=0.0002, beta=0.0004, fc1=512, fc2=256)


num_of_episodes = 100
num_of_iterations = 100

score_history = np.zeros((num_of_episodes,))
rewards = np.zeros((num_of_episodes, num_of_iterations))
sumrate = np.zeros((num_of_episodes, num_of_iterations))
U1_SINR = np.zeros((num_of_episodes, num_of_iterations))
U2_SINR = np.zeros((num_of_episodes, num_of_iterations))

Old_Avg = 0
obs = env.State()

for ep in range(num_of_episodes):
    score = 0
    obs = env.State()

    if ep < num_of_episodes / 4:
        agent.noise = 0.40
    elif ep < num_of_episodes * 2 / 4:
        agent.noise = 0.20
    elif ep < num_of_episodes * 3 / 4:
        agent.noise = 0.1
    elif ep < num_of_episodes * 3 / 4 + 10:
        agent.noise = 0.05
    else:
        agent.noise = 0

    for iter in range(num_of_iterations):
        action = agent.choose_action(obs)

        new_state, reward, sumrate[ep][iter], SINRs = env.Step(action)

        # if iter == 0 or iter == num_of_iterations - 1:
        #     print("****************************************************************")
        #     print("action: ", np.array(action))
        #     print("state: ", obs)
        #     print("New state: ", new_state)
        #     print("SINR: ", SINRs)
        #     print("****************************************************************")

        agent.remember(obs, action, reward, new_state)
        agent.learn()
        obs = new_state
        score += reward
        rewards[ep][iter] = reward

        U1_SINR[ep][iter] = SINRs[0]
        U2_SINR[ep][iter] = SINRs[1]

    # agent.learn()
    score = score / num_of_iterations
    score_history[ep] = score
    New_Avg = score_history[:ep + 1].mean()

    disp(episod=ep, score=score, score_history=score_history,
            New_Avg=New_Avg, Old_Avg=Old_Avg, SINRs=SINRs, sumrate=sumrate[ep][iter])
    
    ac = np.array(action)
    print(f"Power split factor = {ac[-1 : ][0]: <5.2} | U1 Power = {np.linalg.norm(U1.w) : <5.2} | U2 Power = {np.linalg.norm(U2.w): <5.2}" )

    # obs = env.Reset()
    Old_Avg = New_Avg

plot(score_history=score_history, sumrate=sumrate,
        u1_sinr=U1_SINR, u2_sinr=U2_SINR, mean=False,
        title=f"N = {env.N}, M1 = {env.M1}, M2 = {env.M2}")

# agent.save_models()

print("\n\n\n")
print(np.diag(np.angle(env.Psi1, deg=True)))
print("************************************************************************************")
print(np.diag(np.angle(env.Psi2, deg=True)))
# np.linalg.norm(env.Users[1].w)
# np.angle(env.Users[0].w, deg=True)
print("************************************************************************************")
print(max(sumrate.reshape(num_of_episodes*num_of_iterations,)))


Episode 1   Score -> -16.95     Avg-Score -> -16.95     - Max 😀 U1-SINR ->  1.80    U2-SINR ->  0.34    Sumrate ->  1.91   
Power split factor = 0.48  | U1 Power = 0.48  | U2 Power = 0.52 
Episode 2   Score -> -16.61     Avg-Score -> -16.78     + Max 😀 U1-SINR ->  0.00    U2-SINR ->  3.83    Sumrate ->  2.27   
Power split factor = 0.0   | U1 Power = 0.0   | U2 Power = 1.0  
Episode 3   Score -> -8.49      Avg-Score -> -14.02     + Max 😀 U1-SINR ->  0.00    U2-SINR ->  2.97    Sumrate ->  1.99   
Power split factor = 0.0093 | U1 Power = 0.0093 | U2 Power = 0.99 
Episode 4   Score -> -9.79      Avg-Score -> -12.96     +        U1-SINR ->  0.00    U2-SINR ->  9.01    Sumrate ->  3.32   
Power split factor = 0.0   | U1 Power = 0.0   | U2 Power = 1.0  
Episode 5   Score -> -9.45      Avg-Score -> -12.26     +        U1-SINR ->  0.00    U2-SINR ->  11.05   Sumrate ->  3.59   
Power split factor = 0.02  | U1 Power = 0.02  | U2 Power = 0.98 
Episode 6   Score -> -7.55      Avg-Score -> -11.47

KeyboardInterrupt: 

In [5]:
print(np.diag(np.angle(env.Psi1, deg=True)))
print("************************************************************************************")
print(np.diag(np.angle(env.Psi2, deg=True)))
# np.linalg.norm(env.Users[1].w)
# np.angle(env.Users[0].w, deg=True)
print("************************************************************************************")
print(max(sumrate.reshape(num_of_episodes*num_of_iterations,)))

[ 8.77096597e-02  6.15524347e-01  3.45171366e+00 -1.40334186e-14
  0.00000000e+00]
************************************************************************************
[ 2.11226994  1.16170345  4.19638775 -1.94091082  6.73502281]
************************************************************************************
8.419825913625557


In [8]:
import numpy as np
matrix = np.array([1,2,3])

# matrix.__add__(4)
np.append(matrix, 4)
matrix

array([1, 2, 3])

In [47]:
print(np.linalg.norm(U1.w) + np.linalg.norm(U2.w))

2.0
